In [1]:
import sqlite3
import pandas as pd
import numpy as np
import random

    # db_root = '/mnt/FastDisk/GeJunYao/VLP/databases'
    # annotation_db = 'annotation.db'
    # annotation_meta_db = 'annotation_meta.db'
ANNOTATION_DB = '/mnt/FastDisk/GeJunYao/VLP/databases/annotation.db'

conn = sqlite3.connect(ANNOTATION_DB)

## add prompt template for caption rewrite

In [2]:
DESCRIPTION = "A prompt for LLM without instruction-tuning. This prompt is designed for caption rewrite task with 5 examples shots."

PROMPT = """###Instruction###

Revise a description (caption) of a remote sensing image scene. The scene, spanning approximately 72,253 square meters, is oriented with west on the left and north at the top. The coordinate (0, 0) is positioned at the left-bottom corner. The task is to transform the original sentence while preserving its meaning, into five varied versions. These revisions should differ in length, ranging from 20 to 50 words, and in tone. 
    
###EXAMPLE###

####EXAMPLE1####
Raw:
{example1_raw}
Revised:
{example1_revised}

####EXAMPLE2####
Raw:
{example2_raw}
Revised:
{example2_revised}


####EXAMPLE3####
Raw:
{example3_raw}
Revised:
{example3_revised}


####EXAMPLE4####
Raw:
{example4_raw}
Revised:
{example4_revised}


####EXAMPLE5####
Raw:
{example5_raw}
Revised:
{example5_revised}

###TASK###
Raw: 
{raw}
Revised:
"""

# Type larger than 10 for tasks other than caption genration.
TYPE=11

# c = conn.cursor()
# c.execute("INSERT INTO prompt (description, prompt, type) VALUES (?, ?, ?)", (DESCRIPTION, PROMPT, TYPE))
# # link prompt with annotator
# c.execute("INSERT INTO annotator_prompt (PROMPT, ANNOTATOR) VALUES (?, ?)", (c.lastrowid, 1))
# conn.commit()

In [3]:
# import sqlite3
# create a sqlite database for storing rewrite examples
REWRITES_DB = "/mnt/FastDisk/GeJunYao/VLP/databases/annotation_meta.db"

anno_meta_conn = sqlite3.connect(REWRITES_DB)

# create a table for storing rewrite examples
# SQL_CREATE_REWRITE_RAW_TABLE = """ CREATE TABLE IF NOT EXISTS rewrite_raw (
#                                         id integer PRIMARY KEY AUTOINCREMENT,
#                                         patch_id integer NOT NULL,
#                                         prompt_id integer NOT NULL,
#                                         text text NOT NULL)"""
                                        
# SQL_CREATE_REWRITE_EXAMPLES_TABLE = """ CREATE TABLE IF NOT EXISTS rewrite_examples (
#                                         id integer PRIMARY KEY AUTOINCREMENT,
#                                         rewrite_raw_id integer NOT NULL,
#                                         rewrite_text text NOT NULL,
#                                         FOREIGN KEY (rewrite_raw_id) REFERENCES rewrite_raw(id))"""
                                        
# # create the tables
# anno_meta_conn.execute(SQL_CREATE_REWRITE_RAW_TABLE)
# anno_meta_conn.execute(SQL_CREATE_REWRITE_EXAMPLES_TABLE)

# close the connection
# anno_meta_conn.close()

In [22]:
c = conn.cursor()
c.execute("SELECT * FROM annotation")
annotation_table = pd.DataFrame(c.fetchall(), columns=[desc[0] for desc in c.description])
c.execute("SELECT * FROM annotator_prompt")
annotator_prompt_table = pd.DataFrame(c.fetchall(), columns=[desc[0] for desc in c.description])
c.execute("SELECT * FROM annotation_osm")
annotation_osm_table = pd.DataFrame(c.fetchall(), columns=[desc[0] for desc in c.description])

# load prompt template for rewrite into memory
c.execute("select * from prompt where type = 11")
prompt_template = pd.DataFrame(c.fetchall(), columns=[desc[0] for desc in c.description])

# load rewrite examples into memory
c_meta = anno_meta_conn.cursor()
c_meta.execute('select * from rewrite_raw')
rewrite_raw_table = pd.DataFrame(c_meta.fetchall(), columns=[desc[0] for desc in c_meta.description])
c_meta.execute('select * from rewrite_examples')
rewrite_examples_table = pd.DataFrame(c_meta.fetchall(), columns=[desc[0] for desc in c_meta.description])

# exclude the broken patch ids
annotation_table['patch_valid'] = annotation_table['PATCH'].apply(lambda x: isinstance(x, int))
annotation_table = annotation_table[annotation_table['patch_valid'] == True]
# count the number of duplicate annotations
duplicate_count = annotation_table.groupby(['PATCH']).size().reset_index(name='count')

# keep the patches with less than max_rewrites annotations
valid_patches = duplicate_count[duplicate_count['count'] <= 5]['PATCH']

# sample prefetch_size patches randomly
sampled_patches = np.random.choice(valid_patches, 10, replace=False).tolist()

In [17]:
# annotation_table['patch_broken'] = annotation_table['PATCH'].apply(lambda x: isinstance(x, bytes))
# # TODO: validate the code to see if the recovered data is correct.
# broken_data = annotation_table[annotation_table['patch_broken']==True]
# broken_data['recover'] = broken_data['PATCH'].apply(lambda x: int.from_bytes(x, byteorder='little'))
# broken_data


In [13]:
ori_prompt_id

377410    1
Name: PROMPT, dtype: int64

In [25]:
# find the corresponding annotation according to the patch id
data = sampled_patches.pop()
annotation_row = annotation_table[annotation_table['PATCH'] == data]
annotation_row.loc[:, 'CREATED_AT'] = pd.to_datetime(annotation_row['CREATED_AT'])
# find the oldest annotation as the base for the rewrite
oldest_annotation = annotation_row.loc[annotation_row['CREATED_AT'] == annotation_row['CREATED_AT'].min()]
# # extract the first row as a data series
oldest_annotation = oldest_annotation.iloc[0]
# find the corresponding annotator prompt
ori_prompt_id = oldest_annotation['PROMPT']
# find the rewrites
rewrite_examples = rewrite_raw_table[rewrite_raw_table['prompt_id']==ori_prompt_id].\
                    merge(rewrite_examples_table, left_on='id', right_on='rewrite_raw_id', how='left')
selected_examples = []
for i, table in rewrite_examples.groupby('rewrite_raw_id'):
    row = table.sample(n=1).iloc[0]
    selected_examples.append(row)


In [76]:
prompt_template

,ID,DESCRIPTION,PROMPT,TYPE,CREATED_AT
0,4,A prompt for LLM without instruction-tuning. T...,"""###Instruction###\n\nRevise a description (ca...",11,2024-04-03 10:58:20


In [92]:
# shuffle the examples
random.shuffle(selected_examples)
# sample a prompt template
prompt_id, template = prompt_template.sample(n=1).iloc[0][['ID', 'PROMPT']]
# combine the selected examples and the prompt template
format_kwargs = dict()
for i, example in enumerate(selected_examples):
    example_id = f'example{i+1}'
    format_kwargs[example_id+'_raw'] = example['text']
    format_kwargs[example_id+'_revised'] = example['rewrite_text']
format_kwargs['raw'] = oldest_annotation['ANNOTATION']

formated_prompt = template.format(**format_kwargs)

In [72]:
# get the osm ids related to the given annotation id
anno_id = oldest_annotation['ID']
osm_ids = annotation_osm_table[annotation_osm_table['ANNOTATION'] == anno_id]['OSM_ID'].tolist()

In [74]:
oldest_annotation

ID                                                        303034
PATCH                                                      70346
ANNOTATION     A large industrial area is located at the righ...
NUM_ELEMS                                                      1
ANNOTATOR                                                      1
PROMPT                                                         1
CREATED_AT                                   2024-03-06 23:52:48
patch_valid                                                 True
Name: 303033, dtype: object

In [95]:
# select valid annotator ids
valid_annotators = annotator_prompt_table[annotator_prompt_table['PROMPT']==prompt_id]['ANNOTATOR'].tolist()
# build the task
task = dict(type='annotation_task',
            prompt=formated_prompt,
            osm_ids=osm_ids,
            patch_id=oldest_annotation['PATCH'],
            prompt_id=ori_prompt_id,
            valid_annotator_ids=valid_annotators)



In [96]:
task

{'type': 'annotation_task',
 'prompt': '"###Instruction###\n\nRevise a description (caption) of a remote sensing image scene. The scene, spanning approximately 72,253 square meters, is oriented with west on the left and north at the top. The coordinate (0, 0) is positioned at the left-bottom corner. The task is to transform the original sentence while preserving its meaning, into five varied versions. These revisions should differ in length, ranging from 20 to 50 words, and in tone. \n    \n###EXAMPLE###\n\n####EXAMPLE1####\nRaw:\nA nursing home for seniors is located at the right-top of the image patch. It has an irregular boundary with a normalized size of 0.063 which means it covers less than 10% of the image patch. It is likely a building with social facilities.\nRevised:\nPositioned at the image\'s northeast corner, a nursing home occupies less than 10% of the scene, its boundaries irregular, hinting at social service functions. \n\n####EXAMPLE2####\nRaw:\nThe Yellow Lake State Fo

In [70]:
print(formated_prompt)

"###Instruction###

Revise a description (caption) of a remote sensing image scene. The scene, spanning approximately 72,253 square meters, is oriented with west on the left and north at the top. The coordinate (0, 0) is positioned at the left-bottom corner. The task is to transform the original sentence while preserving its meaning, into five varied versions. These revisions should differ in length, ranging from 20 to 50 words, and in tone. 
    
###EXAMPLE###

####EXAMPLE1####
Raw:
The Yellow Lake State Forest, a protected area, is located at the bottom-center of the image patch. It has a rectangular shape and occupies 0.546 of the image patch. It is accessible to the public and has a boundary that is protected. It is a nature reserve and has a protection class of 6. It is related to the New York State Constitution, Article XIV. The source of the information is http://gis.ny.gov/gisdata/inventories/details.cfm?DSID=1114.
Revised:
The image prominently features Yellow Lake State Fores

In [ ]:
####EXAMPLE1####
# Raw:
# {example1_raw}
# Revised:
# {example1_revised}

In [60]:
random.shuffle(selected_examples)

In [56]:
import random
# shuffle the a list
lst = [1, 2, 3, 4, 5]
random.shuffle(lst)
print(lst)

[3, 1, 4, 2, 5]


In [25]:
ori_prompt_id = oldest_annotation['PROMPT']
rewrite_example_raw = rewrite_raw_table[rewrite_raw_table['prompt_id']==ori_prompt_id] 
rewrite_examples = rewrite_example_raw.merge(rewrite_examples_table, left_on='id', right_on='rewrite_raw_id', how='left')
tables = []
for i, table in rewrite_examples.groupby('rewrite_raw_id'):
    tables.append(table)

# choose a row from the table randomly
# table is a pandas dataframe
test_table = tables[0]
test_row = test_table.sample(n=1)
print(test_row)

In [71]:
oldest_annotation

ID                                                        303034
PATCH                                                      70346
ANNOTATION     A large industrial area is located at the righ...
NUM_ELEMS                                                      1
ANNOTATOR                                                      1
PROMPT                                                         1
CREATED_AT                                   2024-03-06 23:52:48
patch_valid                                                 True
Name: 303033, dtype: object

In [26]:
result=""" ###Instruction###

Revise a description (caption) of a remote sensing image scene. The scene, spanning approximately 72,253 square meters, is oriented with west on the left and north at the top. The coordinate (0, 0) is positioned at the left-bottom corner. The task is to transform the original sentence while preserving its meaning, into five varied versions. These revisions should differ in length, ranging from 20 to 50 words, and in tone. 
    
###EXAMPLE###

####EXAMPLE1####
Raw:
A nursing home for seniors is located at the right-top of the image patch. It has an irregular boundary with a normalized size of 0.063 which means it covers less than 10% of the image patch. It is likely a building with social facilities.
Revised:
The image patch reveals, at its right-top, a nursing home for seniors, distinguished by its non-standard perimeter and occupying a mere fraction under 10%, indicative of a place rich in social facilities. 

####EXAMPLE2####
Raw:
A forested area is located at the left-bottom of the image patch. It covers a small portion of the image patch and is likely to be managed for timber production. It is likely to be part of a larger forested area.
Revised:
At the left-bottom corner of the scene, a compact forested area is visible, suggesting it's a segment of a larger forest, possibly cultivated for timber.


####EXAMPLE3####
Raw:
The largest area in the image patch is a rectangular campground named Scaroon Manor Campground. It occupies almost the entire image patch. It is protected by the New York State Constitution, Article XIV. It is a camping site with religious significance. It is likely to have tents, camper vans, or caravans. There could be other recreational activities taking place in the area.
Revised:
The image vividly captures Scaroon Manor Campground, sprawling across most of the frame, safeguarded under New York State Constitution's Article XIV. This campsite, imbued with religious importance, is a haven for tents, camper vans, and possibly a hub for various leisure activities, reflecting a vibrant community spirit.


####EXAMPLE4####
Raw:
A commercial area is located at the bottom-center of the image patch. It has an irregular boundary with a normalized size of 0.311 which means it covers more than one-third of the image patch. It is likely to have buildings and other structures.
Revised:
At the bottom-center of the image, an expansive commercial district emerges, covering more than 30% of the scene with its uneven edges, hinting at the presence of various structures.


####EXAMPLE5####
Raw:
The Yellow Lake State Forest, a protected area, is located at the bottom-center of the image patch. It has a rectangular shape and occupies 0.546 of the image patch. It is accessible to the public and has a boundary that is protected. It is a nature reserve and has a protection class of 6. It is related to the New York State Constitution, Article XIV. The source of the information is http://gis.ny.gov/gisdata/inventories/details.cfm?DSID=1114.
Revised:
Yellow Lake State Forest, identified at the image's bottom-center, occupies 54.6% of the area as a rectangular, Class 6 protected nature reserve, according to New York State Constitution, Article XIV.

###TASK###
Raw: 
A residential area named Partridge Creek is located at the center of the image patch. It has a rectangular shape and occupies 0.963 of the image patch. It is likely to have houses, apartment buildings, and other residential structures.
Revised:
The image patch reveals, at its center, a residential area named Partridge Creek, occupying 96.3% of the scene, likely to be home to houses, apartment buildings, and other residential structures.


###TASK###
Raw:
A residential area named Partridge Creek is located at the center of the image patch. It has a rectangular shape and occupies 0.963 of the image patch. It is likely to have houses, apartment buildings, and other residential structures.
Revised:
The image patch reveals, at its center, a residential area named Partridge Creek, occupying 96.3% of the scene, likely to be home to houses, apartment buildings, and other residential structures.


###TASK###
Raw:
A residential area named Partridge Creek is located at the center of the image patch. It has a rectangular shape and occupies 0. """

In [ ]:
    # task_split = annotation.split("###Task###")
    # pure_text = task_split[1].split('Revised:\n')[1].strip()

In [29]:
task_split = result.split("###TASK###")
task_split[1].split('Revised:\n')[1].strip()

'The image patch reveals, at its center, a residential area named Partridge Creek, occupying 96.3% of the scene, likely to be home to houses, apartment buildings, and other residential structures.'